### RB using Fantasy Pros stats

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

The bulk of the preprocessing logic can be found in the source code.

##### To make our dataset, lets pull running backs from the last ~10 years

In [4]:
RB_2021 = football.prepare_RB("2021")
RB_2020 = football.prepare_RB("2020")
RB_2019 = football.prepare_RB("2019")
RB_2018 = football.prepare_RB("2018")
RB_2017 = football.prepare_RB("2017")
RB_2016 = football.prepare_RB("2016")
RB_2015 = football.prepare_RB("2015")
RB_2014 = football.prepare_RB("2014")
RB_2013 = football.prepare_RB("2013")

In [8]:
all_RB = [RB_2020, RB_2019, RB_2018,
          RB_2017, RB_2016, RB_2015, RB_2014, RB_2013]

RB = pd.concat(all_RB, ignore_index=True)
RB

,ATT,Rushing_Yds,Y/A,LG,20+,Rushing_Td,REC,TGT,Receiving_Yds,Y/R,Receiving_Td,FPTS/G,y,YBCON,YBCON/ATT,YACON,YACON/ATT,BRKTKL,TK LOSS,TK LOSS YDS,LNG TD,10+ YDS,30+ YDS,40+ YDS,50+ YDS,RZ TGT,YACON.1
0,1.000000,1.000000,0.914286,0.959184,1.000000,0.929688,0.214608,0.271612,0.141369,0.419580,0.000000,0.933333,0.894977,1.000000,0.657895,1.000000,0.725000,0.901515,0.862500,0.000000,0.959184,0.913043,1.000000,0.875000,0.500000,0.066964,0.228545
1,0.527690,0.490714,0.857143,0.500000,0.462222,0.933333,1.000000,1.000000,1.000000,0.636364,1.000000,0.938095,0.780822,0.593721,0.736842,0.400246,0.550000,0.537374,0.560000,0.353005,0.408163,0.547826,0.533333,0.466667,0.000000,1.000000,0.975124
2,0.943311,0.877927,0.857143,0.714286,0.457143,1.000000,0.567986,0.540721,0.511621,0.573427,0.214286,1.000000,0.703196,0.967498,0.684211,0.799578,0.600000,1.000000,0.514286,0.606557,0.714286,1.000000,0.571429,0.250000,0.285714,0.153061,0.613006
3,0.654674,0.615364,0.857143,0.632653,0.533333,0.641667,0.433735,0.373832,0.395503,0.580420,0.200000,0.690476,0.607306,0.777707,0.789474,0.473198,0.525000,0.424242,0.800000,0.318033,0.632653,0.710145,0.400000,0.700000,0.533333,0.285714,0.398010
4,0.607710,0.622653,0.928571,0.785714,0.647619,0.562500,0.606713,0.630841,0.503118,0.531469,0.428571,0.719048,0.671233,0.616780,0.657895,0.627410,0.750000,0.424242,0.600000,0.400468,0.785714,0.565217,0.571429,1.000000,0.857143,0.765306,0.458422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009381,0.112903,0.000000,0.000000,0.000000,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
944,0.018817,0.011201,0.103448,0.112500,0.000000,0.000000,0.103896,0.099159,0.139373,0.503226,0.133929,0.000000,0.000000,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
945,0.025090,0.009245,0.065517,0.050000,0.000000,0.000000,0.371058,0.370879,0.381552,0.425806,0.000000,0.000000,0.000000,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000
946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009381,0.112903,0.000000,0.000000,0.000000,0.244472,0.175704,0.263205,0.298218,0.000000,0.295854,0.749949,0.195283,0.225664,0.177985,0.139913,0.088698,0.164551,0.000000


In [9]:
# Dean's feature selection:

y = RB['y']
X = RB.drop(['y', 'FPTS/G'], axis=1)

info = football.getBestFeatures(X, y)
info

{'mean mse': 0.014099398904512514,
 'model': LinearRegression(),
 'features': Index(['ATT', 'Rushing_Yds', 'Rushing_Td', 'YBCON', '10+ YDS'], dtype='object'),
 'coefficients': array([-0.00490778,  0.55285708,  0.35343768, -0.03040537, -0.05665558])}

In [10]:
coefficients = info["coefficients"]
features = info['features']
test_y = RB_2021['y']
test_x = RB_2021[features]

In [14]:
values = test_x.mul(coefficients)
predictions = values.sum(axis = 1)

In [17]:
results = pd.DataFrame()
results['actual'] = test_y
results['predictions'] = predictions
results['difference'] = results['actual'] - results['predictions']

In [18]:
results.loc[:, 'difference'].mean()

0.16265592787991442